# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import random
import requests
import io
import re

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
url = "https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('UTF8')))
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [3]:
# Проверка типов данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Показатель         14 non-null     object
 1   Единица измерений  14 non-null     object
 2   Результат анализа  14 non-null     object
 3   Норматив           14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


In [4]:
df['Результат анализа'] = df['Результат анализа'].apply(lambda x: x.replace(',', '.')) # Замена запятых на точки
df['Результат анализа'] = df['Результат анализа'].apply(lambda x: re.findall(r'\d+\.\d+|\d+', x)) # Поиск чисел. Ожидается список с 1 элементом
df['Результат анализа'] = df['Результат анализа'].apply(lambda x: float(x[0]) if len(x) == 1 else None) # Конвертация в формат float
df['Результат анализа'] = df['Результат анализа'].fillna(0) # Заполнение единственного пропуска нулевым значением.

Единственное нечисловое значение столбца "Реузльтат анализа" в параметре "Цветность"

Цветность — это интенсивность окраски воды из-за наличия цветения, органических соединений, трехвалентного железа. Чтобы определить цветность, образец воды сравнивают с эталоном — 1000 градусной шкалой образцов окраски.

Таким образом, значение "б/цвета" интерпретируем как 0.

In [5]:
def check_water(fact_value: float, norm_string: str) -> str:
    norm_string = norm_string.replace(',', '.')
    criteria = re.findall(r'не более|в пределах', norm_string)[0]
    norm_values = list(map(float, re.findall(r'\d+\.\d+|\d+', norm_string)))
    if criteria == 'не более':
        if fact_value < max(norm_values):
            return 'OK'
        else:
            return 'Нарушение'
    elif criteria == 'в пределах':
        if max(norm_values) >= fact_value >= min(norm_values):
            return 'OK'
        else:
            return 'Нарушение'
    else:
        return None

In [6]:
df['Вывод'] = df.apply(lambda x: check_water(x['Результат анализа'], x['Норматив']), axis=1)

In [7]:
df.set_index(keys='Показатель', inplace=True)

In [8]:
df

,Единица измерений,Результат анализа,Норматив,Вывод
Показатель,,,,
pH,единицы pH,8.400,в пределах 6-9,OK
Запах,баллы,1.000,не более 2-3,OK
Цветность,градусы,0.000,не более 30,OK
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,OK
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",OK
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",OK
Нитраты (по NO3),мг/дм3,24.000,не более 45,OK
Фосфаты (P),мг/дм3,0.360,"не более 3,5",OK
Хлориды (Cl),мг/дм3,200.000,не более 350,OK


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [9]:
fruits = ['apple'] * 4 + ['orange'] * 5
random.shuffle(fruits)
success_cases = 0
total_cases = 10000

for _ in range(total_cases):
    chosen_fruits = random.sample(fruits, 3)
    if chosen_fruits.count('orange') == 3:
        success_cases += 1
print(f'Количество успешных исходов: {success_cases}')
print(f'Количество попыток: {total_cases}')
print(f'Вероятность успешного исхода: {round(success_cases/total_cases, 3)}')

Количество успешных исходов: 1194
Количество попыток: 10000
Вероятность успешного исхода: 0.119


Эксперимент подтвердил аналитическое решение

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [10]:
success_cases = 0
total_cases = 10000

for _ in range(total_cases):
    details = ['standard'] * 7 + ['non-standard'] * 3
    random.shuffle(details )
    checked_detais = 0
    while checked_detais < 10:
        current_indexes = list(range(len(details)))
        current_detail = details.pop(random.choice(current_indexes))
        checked_detais += 1
        if current_detail == 'standard':
            break
    if checked_detais == 2:
        success_cases += 1

print(f'Количество успешных исходов: {success_cases}')
print(f'Количество попыток: {total_cases}')
print(f'Вероятность успешного исхода: {round(success_cases/total_cases, 3)}')

Количество успешных исходов: 2325
Количество попыток: 10000
Вероятность успешного исхода: 0.233


Эксперимент подтвердил аналитическое решение